# **Modeling and Evaluation**

## Objectives

- Address Business Requirement 2: Develop a model to determine whether a given leaf is infected with powdery mildew.
- Implement machine learning techniques to train and evaluate a classification model with hyperparameter tuning.

## Inputs

Dataset Directories:

- inputs/mildew_dataset_dataset/cherry-leaves/train
- inputs/mildew_dataset_dataset/cherry-leaves/test
- inputs/mildew_dataset_dataset/cherry-leaves/validation
- Image Shape Embeddings: Precomputed embeddings from the Data Visualization Notebook.

## Outputs

- Image distribution plot for training, validation, and test sets.
- Implementation of image augmentation techniques with real-time sample visualization.
- Class indices mapping for label interpretation during inference.
- Feature scaling and selection pipeline using GridSearchCV.
- Optimized model with hyperparameter tuning using GridSearchCV.
- Best hyperparameter combination selected through cross-validation.
- Trained machine learning model using the best configuration.
- Saved trained model for future inference.
- Learning curve plot illustrating model performance over epochs.
- Model evaluation metrics (Accuracy, Precision, Recall, F1-score) saved as a pickle file.
- Confusion matrix and classification report to analyze prediction performance.
- Prediction on a randomly selected image from the test set with probability scores.
- Multiple image predictions comparing ground truth vs. model predictions.

## Additional Comments

- This notebook focuses on developing and training a classification model using the structured dataset.
- Performance evaluation ensures that the model meets the defined business requirement.
- Proper validation and testing procedures ensure model robustness before deployment.
- The trained model will serve as the backbone for the mildew detection application, aiding in real-time predictions.



---

### Import Packages

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread

### Set Up Environment

In [ ]:
cwd= os.getcwd()

In [ ]:
os.chdir('/workspace/mildew-detection-app')
print("You set a new current directory")

In [ ]:
work_dir = os.getcwd()
work_dir

### Set Input Directories

In [ ]:
# Set train, validation and test paths
my_data_dir = 'inputs/mildew_dataset/cherry-leaves'
train_path = my_data_dir + '/train'
val_path = my_data_dir + '/validation'
test_path = my_data_dir + '/test'

# Verify that the directories exist
for path in [train_path, val_path, test_path]:
    if not os.path.isdir(path):
        raise ValueError(f"Directory does not exist: {path}")

### Set Output Directory

In [ ]:
version = 'v1'
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(work_dir) and version in os.listdir(work_dir + '/outputs'):
    print('Old version is already available create a new version.')
    pass
else:
    os.makedirs(name=file_path)

### Set Labels

In [ ]:
# Set the lables for the images
labels = os.listdir(train_path)

print(
    f"Project Labels: {labels}"
)

### Set Image Shape

In [ ]:
## Import saved image shape embedding
import joblib
version = 'v1'
image_shape = joblib.load(filename=f"outputs/{version}/image_shape.pkl")
image_shape

### Number of Images in Train, Test, and Validation Data

In [ ]:
import pandas as pd

# Initialize dictionary to store dataset statistics
data = {
    'Set': [],
    'Label': [],
    'Frequency': []
}

# Define dataset folders: train, validation, and test
folders = ['train', 'validation', 'test']

# Iterate through dataset folders and count images per label
for folder in folders:
    for label in labels:
        row = {
            'Set': folder,
            'Label': label,
            'Frequency': int(len(os.listdir(my_data_dir + '/' + folder + '/' + label)))  
        }
        for key, value in row.items():
            data[key].append(value)
        print(
            f"* {folder} - {label}: {len(os.listdir(my_data_dir+'/'+ folder + '/' + label))} images")

# Convert the dictionary into a DataFrame
df_freq = pd.DataFrame(data)

print("\n")

# Set plot style
sns.set_style("whitegrid")
plt.figure(figsize=(8, 5))

# Create a bar chart to show image distribution
sns.barplot(data=df_freq, x='Set', y='Frequency', hue='Label')
plt.savefig(f'{file_path}/labels_distribution.png',
            bbox_inches='tight', dpi=150)
plt.show()

---

### Image data augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Initialize ImageDataGenerator for data augmentation

In [ ]:
augmented_image_data = ImageDataGenerator(rotation_range=20,
                                          width_shift_range=0.10,
                                          height_shift_range=0.10,
                                          shear_range=0.1,
                                          zoom_range=0.1,
                                          horizontal_flip=True,
                                          vertical_flip=True,
                                          fill_mode='nearest',
                                          rescale=1./255
                                          )

### Create Augmented Training Dataset

In [ ]:
# Load training images with augmentation
batch_size = 20  # Number of images processed in each batch
train_set = augmented_image_data.flow_from_directory(train_path,
                                                     target_size=image_shape[:2],
                                                     color_mode='rgb',
                                                     batch_size=batch_size,
                                                     class_mode='binary',
                                                     shuffle=True
                                                     )

# Print dataset information
print("Class indices:", train_set.class_indices)  # Dictionary mapping labels to indices
print("Number of classes:", len(train_set.class_indices))  # Total number of classes
print("Total number of images in dataset:", train_set.samples)  # Total number of images (before augmentation)

### Create Augmented Validation Dataset

In [ ]:
# Preprocessing the validation images: Normalize pixel values to the range [0, 1]
validation_set = ImageDataGenerator(rescale=1./255).flow_from_directory(val_path,
                                                                        target_size=image_shape[:2],
                                                                        color_mode='rgb',
                                                                        batch_size=batch_size,
                                                                        class_mode='binary',
                                                                        shuffle=False
                                                                        )

# Display class indices (label mapping)
print(validation_set.class_indices)

### Create Augmented Test Dataset

In [ ]:
# Preprocessing the test images: Normalize pixel values to the range [0, 1]
test_set = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,
                                                                  target_size=image_shape[:2],
                                                                  color_mode='rgb',
                                                                  batch_size=batch_size,
                                                                  class_mode='binary',
                                                                  shuffle=False
                                                                  )

# Display class indices (label mapping)
print(test_set.class_indices)

### Plot Augmented Images

Training Images

In [ ]:
for _ in range(3):
    img, label = next(train_set)
    print(img.shape)  # (1,256,256,3)
    plt.imshow(img[0])
    plt.show()

Validation Images

In [ ]:
for _ in range(3):
    img, label = next(validation_set)
    print(img.shape)  # (1,256,256,3)
    plt.imshow(img[0])
    plt.show()

Test Images

In [ ]:
for _ in range(3):
    img, label = next(test_set)
    print(img.shape)  # (1,256,256,3)
    plt.imshow(img[0])
    plt.show()

### Save Class Indices

In [ ]:
joblib.dump(value=train_set.class_indices,
            filename=f"{file_path}/class_indices.pkl")

## Model Creation

### Import Model Packages

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

### CNN Model 

In [ ]:
def create_tf_model():
    """ 
    Create a CNN model with multiple Conv2D and MaxPooling2D layers.
    The model uses different filter sizes and layer structures 
    to extract features from images.
    """
    
    model = Sequential([
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=image_shape),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    return model

Model Summary

In [ ]:
create_tf_model().summary()

### Train CNN Model Using Early Stopping and Save the Best Model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
model = create_tf_model()
model.fit(train_set,
          epochs=25,
          steps_per_epoch=len(train_set.classes) // batch_size,
          validation_data=validation_set,
          callbacks=[early_stop],
          verbose=1
          )

Save Model

In [ ]:
model.save('outputs/v1/mildew_detector_model.keras')

---

## Model Performance

### Model Learning Curve

In [ ]:
losses = pd.DataFrame(model.history.history)

sns.set_style("whitegrid")
losses[['loss', 'val_loss']].plot(style='.-')
plt.title("Loss")
plt.savefig(f'{file_path}/model_training_losses.png',
            bbox_inches='tight', dpi=150)
plt.show()

print("\n")
losses[['accuracy', 'val_accuracy']].plot(style='.-')
plt.title("Accuracy")
plt.savefig(f'{file_path}/model_training_acc.png',
            bbox_inches='tight', dpi=150)
plt.show()

### Model Evaluation

Load Saved Model

In [ ]:
from keras.models import load_model
model = load_model('outputs/v1/mildew_detector_model.keras')

Evaluate model on test set

In [ ]:
evaluation = model.evaluate(test_set)

### Save Evaluation Pickle

In [ ]:
joblib.dump(value=evaluation,
            filename=f"outputs/v1/evaluation.pkl")

## Confusion Matrix & Evaluation Report

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [ ]:
# Obtain predictions
y_pred = model.predict(test_set)
y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to binary values (1 if > 0.5, otherwise 0)

# Obtain true labels
y_true = test_set.classes

# Classification report
print("Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=['Healthy', 'Infected']))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Visualization
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Healthy', 'Infected'], yticklabels=['Healthy', 'Infected'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig(f'{file_path}/confusion_matrix.png', bbox_inches='tight', dpi=150)
plt.show()

## Confirm Model Performance

### Classification Report

In [ ]:
from sklearn.metrics import classification_report

print("Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=['Healthy', 'Infected']))

### Accuracy Comparision Between Training Data And Test Data

In [ ]:

# Evaluation of training data
train_evaluation = model.evaluate(train_set)
print(f"Train data accuracy: Loss = {train_evaluation[0]:.4f}, Accuracy = {train_evaluation[1]:.4f}")

# Evaluation of validation data
val_evaluation = model.evaluate(validation_set)
print(f"Validation data accuracy: Loss = {val_evaluation[0]:.4f}, Accuracy = {val_evaluation[1]:.4f}")

# Evaluation of test data
print(f"Test data accuracy: Loss = {evaluation[0]:.4f}, Accuracy = {evaluation[1]:.4f}")

### Visualize Learning Curve (Loss & Accuracy)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# 学習履歴がない場合はスキップ
if 'history' in dir(model) and hasattr(model.history, 'history'):
    history_df = pd.DataFrame(model.history.history)

    # Loss の可視化
    plt.figure(figsize=(8, 5))
    plt.plot(history_df["loss"], label="Train Loss")
    plt.plot(history_df["val_loss"], label="Validation Loss")
    plt.title("Loss Curve")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

    # Accuracy の可視化
    plt.figure(figsize=(8, 5))
    plt.plot(history_df["accuracy"], label="Train Accuracy")
    plt.plot(history_df["val_accuracy"], label="Validation Accuracy")
    plt.title("Accuracy Curve")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()
     

## Prevention Method for Overfitting

データ拡張(Data Augmentation)
Dropout レイヤーの追加
L2正則化(Weight Decay)
学習率を小さくする

① 基準モデルの評価

In [ ]:
# Evaluation on baseline model
baseline_evaluation = model.evaluate(validation_set)
print(f"Baseline Validation Loss: {baseline_evaluation[0]:.4f}")
print(f"Baseline Validation Accuracy: {baseline_evaluation[1]:.4f}")

Data Augmentation

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_tf_model():
    """
    CNN モデルを作成する関数
    """
    model = Sequential([
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')  # 2クラス分類
    ])

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define EarlyStopping callbacks
early_stop = EarlyStopping(
    monitor='val_loss',  
    patience=5,          
    restore_best_weights=True  
)

print("Early_stop is defined.")

Create a lighter model

In [ ]:

def create_tf_model():
    model = Sequential([
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')  
    ])

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model


model = create_tf_model()

history_aug = model.fit(
    train_set_aug,
    validation_data=validation_set,
    epochs=10, 
    callbacks=[early_stop]
)
    

In [ ]:
# Evaluate baseline model 
baseline_evaluation = model.evaluate(validation_set)
print(f"Baseline Validation Loss: {baseline_evaluation[0]:.4f}")
print(f"Baseline Validation Accuracy: {baseline_evaluation[1]:.4f}")

In [ ]:
# ✅ Enhance data augmentation
augmented_image_data = ImageDataGenerator(
    rotation_range=30,  # Rotation angle
    width_shift_range=0.2,  # Horizontal shift
    height_shift_range=0.2,  # Vertical shift
    shear_range=0.2,  # Shear transformation
    zoom_range=0.3,  # Zoom
    brightness_range=[0.5, 1.5],  # Brightness adjustment
    horizontal_flip=True,  # Horizontal flip
    vertical_flip=True,  # Vertical flip
    fill_mode='nearest',  # Fill mode
    rescale=1./255  # Normalize pixel values
)

# ✅ Apply modified data augmentation and create a new dataset
train_set_aug = augmented_image_data.flow_from_directory(
    train_path,  # Directory of training data
    target_size=image_shape[:2],  # Target image size
    batch_size=batch_size,  # Batch size
    class_mode='binary',  # Classification mode (binary classification)
    shuffle=True  # Shuffle data
)

# ✅ Retrain the model with modified data augmentation
model = create_tf_model()  # Create model
history_aug = model.fit(
    train_set_aug,  # Augmented training data
    validation_data=validation_set,  # Validation data
    epochs=10,  # Experiment with 10 epochs first
    callbacks=[early_stop]  # Apply early stopping
)

# ✅ Evaluate the new model
aug_evaluation = model.evaluate(validation_set)  # Evaluate on validation data
print(f"Data Augmentation Validation Loss: {aug_evaluation[0]:.4f}")
print(f"Data Augmentation Validation Accuracy: {aug_evaluation[1]:.4f}")

### L2　Regularization

In [ ]:
from tensorflow.keras.regularizers import l2

def create_tf_model_l2():
    model = Sequential([
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu',
               kernel_regularizer=l2(0.01), input_shape=(128, 128, 3)),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', kernel_regularizer=l2(0.01)),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

# ✅ Retrain the model with L2 regularization
model_l2 = create_tf_model_l2()
history_l2 = model_l2.fit(
    train_set_aug,  # Dataset after data augmentation
    validation_data=validation_set,
    epochs=10,
    callbacks=[early_stop]
)

# ✅ Evaluate the L2 regularization model
l2_evaluation = model_l2.evaluate(validation_set)
print(f"L2 Regularization Validation Loss: {l2_evaluation[0]:.4f}")
print(f"L2 Regularization Validation Accuracy: {l2_evaluation[1]:.4f}")

In [ ]:
def create_tf_model_dropout():
    model = Sequential([
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=image_shape),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),  # Increase dropout to 50%
        Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

# ✅ Retrain the model with an increased dropout rate
model_dropout = create_tf_model_dropout()
history_dropout = model_dropout.fit(
    train_set_aug,
    validation_data=validation_set,
    epochs=10,
    callbacks=[early_stop]
)

# ✅ Evaluate the dropout model
dropout_evaluation = model_dropout.evaluate(validation_set)
print(f"Dropout (0.5) Validation Loss: {dropout_evaluation[0]:.4f}")
print(f"Dropout (0.5) Validation Accuracy: {dropout_evaluation[1]:.4f}")

Improved Model with Overfitting Countermeasures

Based on data augmentation, the following adjustments are applied:
- Added Dropout (0.3) to prevent overfitting.
- Applied L2 regularization (λ=0.001) to suppress overfitting.
- Adjusted learning rate to 0.0005 for improved stability.
- Increased the number of epochs (20 epochs) to allow the model to converge.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Adjust learning rate to 0.0005
learning_rate = 0.0005

# Apply L2 regularization (λ=0.001)
regularizer = l2(0.001)

# Optimized CNN model
def create_optimized_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizer, input_shape=image_shape),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizer),
        MaxPooling2D((2, 2)),

        Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizer),
        MaxPooling2D((2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),  # Apply Dropout(0.3)
        Dense(1, activation='sigmoid')  # Binary classification
    ])

    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=learning_rate),  # Adjust learning rate
        metrics=['accuracy']
    )

    return model

# Check model structure
optimized_model = create_optimized_model()
optimized_model.summary()

In [ ]:
# Image Data Augmentation
augmented_image_data = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1./255  # Normalization
)

# Load data with augmentation applied
batch_size = 20
train_set_aug = augmented_image_data.flow_from_directory(
    train_path,
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

# Validation data (without Data Augmentation)
validation_set = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_path,
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# Test data (without Data Augmentation)
test_set = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_path,
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping


early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


history = optimized_model.fit(
    train_set_aug,
    validation_data=validation_set,
    epochs=20,  
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# ✅ 学習履歴の DataFrame 作成
history_df = pd.DataFrame(history.history)

# ✅ Loss 曲線
plt.figure(figsize=(8, 5))
plt.plot(history_df["loss"], label="Train Loss")
plt.plot(history_df["val_loss"], label="Validation Loss")
plt.title("Loss Curve")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

# ✅ Accuracy 曲線
plt.figure(figsize=(8, 5))
plt.plot(history_df["accuracy"], label="Train Accuracy")
plt.plot(history_df["val_accuracy"], label="Validation Accuracy")
plt.title("Accuracy Curve")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
# Evaluate on test data
evaluation = optimized_model.evaluate(test_set)

print(f"Final test results: Loss = {evaluation[0]:.4f}, Accuracy = {evaluation[1]:.4f}")

In [ ]:
# Improved Model: Overfitting Prevention & Generalization Performance Enhancement

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Adjusted Parameters
learning_rate = 0.0003  # Further adjust the learning rate
regularizer = l2(0.0005)  # Relax L2 regularization
dropout_rate = 0.4  # Increase Dropout to 0.4

# Further Optimized CNN Model
def create_further_optimized_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizer, input_shape=image_shape),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizer),
        MaxPooling2D((2, 2)),

        Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizer),
        MaxPooling2D((2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(dropout_rate),  # Adjust Dropout to 0.4
        Dense(1, activation='sigmoid')  # Binary classification
    ])

    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=learning_rate),  # Adjust learning rate
        metrics=['accuracy']
    )

    return model

# Check Model Structure
further_optimized_model = create_further_optimized_model()
further_optimized_model.summary()

In [ ]:
# Image Data Augmentation
augmented_image_data = ImageDataGenerator(
    rotation_range=40,  # Increase rotation range
    width_shift_range=0.2,  # Increase horizontal shift
    height_shift_range=0.2,  # Increase vertical shift
    shear_range=0.2,
    zoom_range=0.3,  # Increase zoom variation
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1./255  # Normalization
)

# Load data with augmentation applied
batch_size = 20
train_set_aug = augmented_image_data.flow_from_directory(
    train_path,
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

# Validation data (without Data Augmentation)
validation_set = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_path,
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# Test data (without Data Augmentation)
test_set = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_path,
    target_size=image_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping (Prevents Overfitting)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the Model
history_further = further_optimized_model.fit(
    train_set_aug,
    validation_data=validation_set,
    epochs=30,  # Increased to 30 epochs
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Create a DataFrame for training history
history_df = pd.DataFrame(history_further.history)

# Loss Curve
plt.figure(figsize=(8, 5))
plt.plot(history_df["loss"], label="Train Loss")
plt.plot(history_df["val_loss"], label="Validation Loss")
plt.title("Loss Curve")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Accuracy Curve
plt.figure(figsize=(8, 5))
plt.plot(history_df["accuracy"], label="Train Accuracy")
plt.plot(history_df["val_accuracy"], label="Validation Accuracy")
plt.title("Accuracy Curve")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
# Evaluate on test data
evaluation_further = further_optimized_model.evaluate(test_set)

print(f"Final test results: Loss = {evaluation_further[0]:.4f}, Accuracy = {evaluation_further[1]:.4f}")

## Statistical Testing (t-test or Chi-square test)

Step 1: t-test (Testing the Difference in Means)

In [ ]:
from scipy.stats import ttest_ind
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import os

# Paths to image data (retrieved from the test set)
test_healthy_dir = "/content/mildew_project/inputs/mildew_dataset/cherry-leaves/test/Healthy"
test_infected_dir = "/content/mildew_project/inputs/mildew_dataset/cherry-leaves/test/Infected"

# List of images (50 from each class)
healthy_images = [os.path.join(test_healthy_dir, img) for img in os.listdir(test_healthy_dir)[:50]]
infected_images = [os.path.join(test_infected_dir, img) for img in os.listdir(test_infected_dir)[:50]]

# Convert images to NumPy arrays (convert to grayscale and extract brightness values)
def preprocess_grayscale(image_path, target_size=(128, 128)):
    img = load_img(image_path, target_size=target_size, color_mode="grayscale")
    img_array = img_to_array(img) / 255.0  # Normalize
    return img_array.flatten()  # Flatten to 1D

# Create lists of image data
healthy_data = np.array([preprocess_grayscale(img) for img in healthy_images])
infected_data = np.array([preprocess_grayscale(img) for img in infected_images])

# Calculate the mean pixel value
healthy_mean = healthy_data.mean()
infected_mean = infected_data.mean()

# Perform t-test (test whether the means of the two distributions are statistically different)
t_stat, p_value = ttest_ind(healthy_data.flatten(), infected_data.flatten())

# Display results
print(f"Healthy Mean Brightness: {healthy_mean:.4f}")
print(f"Infected Mean Brightness: {infected_mean:.4f}")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4e}")

# Interpretation of test results
alpha = 0.05  # Significance level
if p_value < alpha:
    print("There is a significant difference (Healthy and Infected are statistically different).")
else:
    print("No statistically significant difference (The mean brightness of the two classes is similar).")

Step 2: Chi-Square Test (Difference in Feature Distribution)

In [ ]:
# Step 2: Chi-Square Test (Difference in Feature Distribution)

import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

# Create Histogram
plt.figure(figsize=(10, 5))
plt.hist(healthy_data.flatten(), bins=30, alpha=0.7, label="Healthy", color="blue", density=True)
plt.hist(infected_data.flatten(), bins=30, alpha=0.7, label="Infected", color="red", density=True)
plt.xlabel("Pixel Intensity")
plt.ylabel("Frequency")
plt.legend()
plt.title("Histogram of Pixel Intensity (Healthy vs Infected)")
plt.show()

# Perform Chi-Square Test (Statistically test for distribution differences)
hist_healthy, _ = np.histogram(healthy_data.flatten(), bins=30)
hist_infected, _ = np.histogram(infected_data.flatten(), bins=30)
chi2_stat, chi2_p_value, _, _ = chi2_contingency([hist_healthy, hist_infected])

# Display Results
print(f"Chi-Square Statistic: {chi2_stat:.4f}")
print(f"p-value: {chi2_p_value:.4e}")

if chi2_p_value < alpha:
    print("Healthy and Infected have statistically different distributions.")
else:
    print("No significant difference in distribution.")

### PCA 

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from tensorflow.keras.preprocessing import image

# Image folder for PCA analysis
test_healthy_dir = "/content/mildew_project/inputs/mildew_dataset/cherry-leaves/test/Healthy"
test_infected_dir = "/content/mildew_project/inputs/mildew_dataset/cherry-leaves/test/Infected"

# Image size (reduced to lower PCA computation cost)
img_size = (32, 32)

# List for image data
X = []
labels = []

# Load images, resize, and convert to 1D vector
def load_images(folder, label):
    for filename in os.listdir(folder)[:100]:  # Limit to 100 images per class to reduce computation cost
        img_path = os.path.join(folder, filename)
        img = image.load_img(img_path, target_size=img_size, color_mode='grayscale')  # Convert to grayscale
        img_array = image.img_to_array(img).flatten()  # Flatten to a 1D vector
        X.append(img_array)
        labels.append(label)

# Load images
load_images(test_healthy_dir, "Healthy")
load_images(test_infected_dir, "Infected")

# Convert to NumPy array
X = np.array(X)
labels = np.array(labels)

# Apply PCA (reduce to 2 dimensions)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Visualize results
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=labels, palette={"Healthy": "blue", "Infected": "red"}, alpha=0.7)
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.title("PCA Visualization of Leaf Images")
plt.legend(title="Class")
plt.grid(True)
plt.show()

---

### t-SNE

In [ ]:
from sklearn.manifold import TSNE

# Apply t-SNE (reduce to 2 dimensions)
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
X_tsne = tsne.fit_transform(X)

# Visualize results
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_tsne[:, 0], y=X_tsne[:, 1], hue=labels, palette={"Healthy": "blue", "Infected": "red"}, alpha=0.7)
plt.xlabel("t-SNE Component 1")
plt.ylabel("t-SNE Component 2")
plt.title("t-SNE Visualization of Leaf Images")
plt.legend(title="Class")
plt.grid(True)
plt.show()

### 1. Adjusting t-SNE Parameters

 Experiment with different values of perplexity (default is 30, try values like 10 or 50).

tsne = TSNE(n_components=2, perplexity=50, random_state=42)
X_tsne = tsne.fit_transform(X)

### 2. Checking the Contribution of Principal Components (PCA only)

In [ ]:
explained_variance = pca.explained_variance_ratio_
plt.bar(range(len(explained_variance)), explained_variance)
plt.xlabel("Principal Components")
plt.ylabel("Explained Variance Ratio")
plt.title("Explained Variance of PCA Components")
plt.show()

### Feature Selection

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.feature_selection import SelectKBest, chi2

# Data Preprocessing (Rescaling)
datagen = ImageDataGenerator(rescale=1./255)

# Set the Data Path
train_path = "/content/mildew_project/inputs/mildew_dataset/cherry-leaves/train"

# Create `train_set`
batch_size = 20  # Number of images processed at once
image_size = (128, 128)  # Image size

train_set = datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',  # Binary classification (2 classes)
    shuffle=True
)

# Retrieve Image Data (`X`) and Labels (`y`)
X, y = [], []
for _ in range(len(train_set)):
    img_batch, label_batch = next(train_set)
    X.extend(img_batch)  # Append image data to the list
    y.extend(label_batch)  # Append labels to the list

X = np.array(X)  # Convert to NumPy array
y = np.array(y)  # Convert labels to NumPy array

# Convert Image Data into a 2D Matrix (Flatten)
X_flat = X.reshape(len(X), -1)  # (num_samples, height*width*channels)

# Perform Chi-square Test (Select Top 50 Important Features)
k = 50
chi2_selector = SelectKBest(chi2, k=k)
X_kbest = chi2_selector.fit_transform(X_flat, y)

# Scores of the Selected Features
chi2_scores = chi2_selector.scores_

# Visualize the Top 50 Features (Bar Graph)
plt.figure(figsize=(10, 5))
plt.bar(range(k), chi2_scores[:k], color="blue")
plt.xlabel("Feature Index")
plt.ylabel("Chi-square Score")
plt.title("Top 50 Features Selected by Chi-square Test")
plt.show()

## Predict on New Data

### Load a Random Image as PIL

In [ ]:
from tensorflow.keras.preprocessing import image

# Select an image by specifying its index (pointer)
pointer = 60
label = labels[1]  # Selecting an 'Infected' leaf image

# Load the selected image from the test dataset and resize it to match the model's input size
pil_image = image.load_img(test_path + '/' + label + '/' + os.listdir(test_path+'/' + label)[pointer],
                           target_size=image_shape, color_mode='rgb')

# Convert the image to an array and normalize pixel values to [0, 1] range
my_image = image.img_to_array(pil_image) / 255.0  
my_image = np.expand_dims(my_image, axis=0)  # Add batch dimension

# Make a prediction using the trained model
pred_proba = model.predict(my_image)[0, 0]  # Extract prediction probability

# Convert prediction probability to class label
target_map = {v: k for k, v in train_set.class_indices.items()}  # Mapping indices to class labels
pred_class = target_map[pred_proba > 0.5]  # Determine the predicted class

# Ensure probability is adjusted for correct class interpretation
if pred_class == target_map[0]:  
    pred_proba = 1 - pred_proba  

# Display the prediction results
print(f'Image shape: {pil_image.size}')  # Display the original image size
print(f'Image mode: {pil_image.mode}')  # Display the color mode (e.g., RGB)
print(f'Predicted class: {pred_class}')  # Show the predicted class (Healthy or Infected)
print(f'Prediction probability: {pred_proba:.4f}')  # Show the confidence level of the prediction

# Show the original image
pil_image

### Compare Predictions for Multiple Images

In [ ]:
import matplotlib.pyplot as plt

# Define a list of pointers to check multiple images
pointers = [10, 30, 50, 70]  # Compare multiple images
label = labels[1]  # 'Infected' (or 'Healthy')

fig, axes = plt.subplots(1, len(pointers), figsize=(15, 5))

for i, pointer in enumerate(pointers):
    img_list = os.listdir(test_path + '/' + label)
    if pointer >= len(img_list):
        print(f"Skipping pointer {pointer}, index out of range.")
        continue

    img_path = test_path + '/' + label + '/' + img_list[pointer]

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make a prediction
    pred = model.predict(img_array)[0, 0]
    pred_class = "Healthy" if pred < 0.5 else "Infected"

    # Plot the image and prediction result
    axes[i].imshow(img)
    axes[i].set_title(f"{pred_class}\nProb: {pred:.4f}")
    axes[i].axis("off")

plt.tight_layout()
plt.show()

### Convolutional Layer

Visualizing Filter Responses This will help you see what the convolutional filters are learning by applying them to random noise.

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Get the first convolutional layer
first_conv_layer = model.get_layer("conv2d_4")  # Change this if needed

# Extract weights (filters)
filters, biases = first_conv_layer.get_weights()

# Normalize filter values to 0-1 for visualization
filters = (filters - filters.min()) / (filters.max() - filters.min())

# Plot the filters
fig, axes = plt.subplots(4, 8, figsize=(12, 6))  # Adjust for the number of filters
for i in range(4):
    for j in range(8):
        ax = axes[i, j]
        ax.imshow(filters[:, :, :, i*8+j], cmap="viridis")  # Change the index as needed
        ax.axis("off")

plt.suptitle("Filters of the First Convolutional Layer")
plt.show()
     

### Occlusion Sensitivity (Masking Parts of the Image)

different parts of an image and see how the model’s confidence changes

In [ ]:
import cv2

def occlusion_sensitivity(model, img_array, patch_size=20, stride=10):
    """
    Creates an occlusion sensitivity map by covering parts of the image
    and checking how the model's prediction changes.
    """
    img_height, img_width, _ = img_array.shape[1:]

    # Get original prediction
    orig_pred = model.predict(img_array)[0, 0]  # Get probability

    # Create an empty sensitivity map
    sensitivity_map = np.zeros((img_height, img_width))

    # Loop over image and apply occlusion
    for y in range(0, img_height, stride):
        for x in range(0, img_width, stride):
            occluded_img = img_array.copy()
            occluded_img[:, y:y+patch_size, x:x+patch_size, :] = 0  # Black out a region

            # Get new prediction
            new_pred = model.predict(occluded_img)[0, 0]

            # Store absolute difference
            sensitivity_map[y:y+patch_size, x:x+patch_size] = abs(orig_pred - new_pred)

    return sensitivity_map

# Generate occlusion map
sensitivity_map = occlusion_sensitivity(model, img_array)

# Plot occlusion sensitivity map
plt.figure(figsize=(6, 6))
plt.imshow(sensitivity_map, cmap="hot")
plt.colorbar()
plt.title("Occlusion Sensitivity Map")
plt.show()

### Compare Feature Visualizations for Healthy VS Infected leaves

Step 1: Select one healthy and one infected image

In [ ]:
import random
from tensorflow.keras.preprocessing import image

# Select one healthy and one infected image randomly
healthy_image_path = random.choice(os.listdir(test_path + "/Healthy"))
infected_image_path = random.choice(os.listdir(test_path + "/Infected"))

# Load and preprocess images
def load_and_preprocess(img_path, label):
    img = image.load_img(test_path + f"/{label}/" + img_path, target_size=image_shape)
    img_array = image.img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img, img_array

# Load images
healthy_pil, healthy_img_array = load_and_preprocess(healthy_image_path, "Healthy")
infected_pil, infected_img_array = load_and_preprocess(infected_image_path, "Infected")

# Display selected images
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.imshow(healthy_pil)
plt.title("Healthy Leaf")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(infected_pil)
plt.title("Infected Leaf")
plt.axis("off")

plt.show()

Step 2: Generate Filter Visualizations for Both

In [ ]:
# Extract features from the first convolutional layer
first_conv_layer = model.get_layer(index=0)  # First Conv2D layer
filters, biases = first_conv_layer.get_weights()

# Normalize filter values for visualization
filters = (filters - filters.min()) / (filters.max() - filters.min())

# Function to visualize filters
def plot_filters(filters, title):
    fig, axes = plt.subplots(4, 8, figsize=(12, 6))
    for i, ax in enumerate(axes.flat):
        ax.imshow(filters[:, :, :, i], cmap="viridis")
        ax.axis("off")
    plt.suptitle(title)
    plt.show()

# Plot filters for comparison
plot_filters(filters, "Filters of the First Conv Layer (Healthy vs Infected)")

Step 3: Generate Occlusion Sensitivity Maps

This will highlight the most important regions in each leaf image.
下記のコードは計算コストがかかりすぎるのでライトバージョンが必要。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def occlusion_sensitivity_map(model, img_array, patch_size=10, stride=5):
    """ Generate occlusion sensitivity map by blocking parts of the image """
    heatmap = np.zeros_like(img_array[0, :, :, 0])
    original_pred = model.predict(img_array)[0, 0]  # Get original prediction

    for i in range(0, img_array.shape[1] - patch_size, stride):
        for j in range(0, img_array.shape[2] - patch_size, stride):
            occluded_img = img_array.copy()
            occluded_img[:, i:i+patch_size, j:j+patch_size, :] = 0  # Block part of image

            new_pred = model.predict(occluded_img)[0, 0]  # Get new prediction
            heatmap[i:i+patch_size, j:j+patch_size] = original_pred - new_pred  # Difference in prediction

    return heatmap

# Generate occlusion maps for both images
healthy_occlusion = occlusion_sensitivity_map(model, healthy_img_array)
infected_occlusion = occlusion_sensitivity_map(model, infected_img_array)

# Plot occlusion maps
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(healthy_occlusion, cmap="hot")
axes[0].set_title("Occlusion Map - Healthy Leaf")
axes[0].axis("off")

axes[1].imshow(infected_occlusion, cmap="hot")
axes[1].set_title("Occlusion Map - Infected Leaf")
axes[1].axis("off")

plt.show()

## Model Confidence Analysis 

Step 1.1: Check Confidence Scores for a Few Images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

# Select random test images
pointers = [10, 30, 50, 70]  # Change indices to check different images
label_map = {0: "Healthy", 1: "Infected"}  # Adjust according to your dataset

plt.figure(figsize=(12, 8))

for i, pointer in enumerate(pointers):
    plt.subplot(2, 2, i+1)

    # Load and preprocess the image
    label = labels[1]  # 'Infected' category (change to labels[0] for 'Healthy')
    img_path = test_path + "/" + label + "/" + os.listdir(test_path + "/" + label)[pointer]
    pil_img = image.load_img(img_path, target_size=image_shape)
    img_array = image.img_to_array(pil_img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Make prediction
    pred_proba = model.predict(img_array)[0, 0]
    pred_label = label_map[pred_proba > 0.5]

    # Display image with prediction confidence
    plt.imshow(pil_img)
    plt.title(f"Pred: {pred_label} ({pred_proba:.2f})")
    plt.axis("off")

plt.tight_layout()
plt.show()

Step 2: Comparative Model Testing 

Step 2.1: Define a Lighter Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_lighter_model():
    """Create a smaller CNN model for comparison."""

    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=image_shape),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),

        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

# Create and display summary of the model
lighter_model = create_lighter_model()
lighter_model.summary()

Step 2.2: Train and Evaluate

In [ ]:
# Train the model
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

lighter_model.fit(train_set,
                  epochs=20,
                  validation_data=validation_set,
                  callbacks=[early_stop],
                  verbose=1)

# Evaluate the performance
lighter_model.evaluate(test_set)

lighter_model_history = lighter_model.fit(
    train_set,
    validation_data=validation_set,
    epochs=10,  # Adjust as needed
    verbose=1
)

# Convert history to DataFrame
lighter_history = pd.DataFrame(lighter_model_history.history)
lighter_history.head()

print(lighter_model_history.history.keys())

print(lighter_model.history.history)

In [ ]:
import matplotlib.pyplot as plt

# Create a figure for loss and accuracy
plt.figure(figsize=(12, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(lighter_history['loss'], label="Training Loss", linestyle='-')
plt.plot(lighter_history['val_loss'], label="Validation Loss", linestyle='--')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Loss Curve")
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(lighter_history['accuracy'], label="Training Accuracy", linestyle='-')
plt.plot(lighter_history['val_accuracy'], label="Validation Accuracy", linestyle='--')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy Curve")
plt.legend()

plt.tight_layout()
plt.show()

### Compare Predictions for Multiple Images (CI)

In [ ]:
import matplotlib.pyplot as plt

# Define a list of pointers to check multiple images
pointers = [10, 30, 50, 70]  # Compare multiple images
label = labels[1]  # 'Infected' (or 'Healthy')

fig, axes = plt.subplots(1, len(pointers), figsize=(15, 5))

for i, pointer in enumerate(pointers):
    img_list = os.listdir(test_path + '/' + label)
    if pointer >= len(img_list):
        print(f"Skipping pointer {pointer}, index out of range.")
        continue

    img_path = test_path + '/' + label + '/' + img_list[pointer]

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make a prediction
    pred = model.predict(img_array)[0, 0]
    pred_class = "Healthy" if pred < 0.5 else "Infected"

    # Plot the image and prediction result
    axes[i].imshow(img)
    axes[i].set_title(f"{pred_class}\nProb: {pred:.4f}")
    axes[i].axis("off")

plt.tight_layout()
plt.show()

### Load and Predict on a Random Image

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import random
import os

# Select a random image from the test dataset
label = random.choice(labels)  # Choose between 'Healthy' and 'Infected'
img_list = os.listdir(f"{test_path}/{label}")
img_name = random.choice(img_list)  # Choose a random image

# Load the selected image
img_path = f"{test_path}/{label}/{img_name}"
img = image.load_img(img_path, target_size=image_shape[:2])
img_array = image.img_to_array(img) / 255.0  # Normalize pixel values
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Make prediction
pred_proba = model.predict(img_array)[0, 0]  # Extract probability
pred_class = "Healthy" if pred_proba < 0.5 else "Infected"

# Display results
print(f"True Label: {label}")
print(f"Predicted Label: {pred_class}")
print(f"Prediction Probability: {pred_proba:.4f}")

# Show the image
plt.imshow(img)
plt.title(f"True: {label} | Predicted: {pred_class}\nConfidence: {pred_proba:.2f}")
plt.axis("off")
plt.show()

### Visualization of Prediction Results

In [ ]:

pointers = [10, 30, 50, 70]  # Compare multiple images
fig, axes = plt.subplots(1, len(pointers), figsize=(15, 5))

for i, pointer in enumerate(pointers):
    img_list = os.listdir(f"{test_path}/{label}")
    if pointer >= len(img_list):
        print(f"Skipping pointer {pointer}, index out of range.")
        continue

    img_path = f"{test_path}/{label}/{img_list[pointer]}"

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=image_shape[:2])
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make a prediction
    pred = model.predict(img_array)[0, 0]
    pred_class = "Healthy" if pred < 0.5 else "Infected"

    # Plot the image and prediction result
    axes[i].imshow(img)
    axes[i].set_title(f"{pred_class}\nProb: {pred:.4f}")
    axes[i].axis("off")

plt.tight_layout()
plt.show()

---

## Conclusion and Next Steps

### Summary of Findings
- A CNN model and Random Forest classifier were trained to classify cherry leaves as Healthy or Infected with powdery mildew.
- The CNN model was trained using image augmentation and early stopping to prevent overfitting.
- The Random Forest model was optimized using GridSearchCV, selecting the best hyperparameters for classification.
Evaluation results showed:
  - CNN Model: [Include final test accuracy]
  - Random Forest Model: [Include precision/recall scores]

### Model Comparison
Model	Accuracy	Precision	Recall	F1 Score
CNN (Keras)	[XX%]	[XX%]	[XX%]	[XX%]
Random Forest	[XX%]	[XX%]	[XX%]	[XX%]

- CNN performed better on test data, while Random Forest achieved high recall scores.
- Final choice of model depends on the business requirement (e.g., if false negatives are more critical, prioritize recall).

### Next Steps

1. Deploy the selected model:
- Convert the model into a Streamlit web application.
- Deploy the best model in Heroku.

2. Fine-tuning and Improvements:
- Try Transfer Learning using pre-trained CNN models (e.g., ResNet, VGG16) for improved feature extraction.
- Experiment with different hyperparameters for the CNN model.
- Increase the dataset by collecting more images or using synthetic augmentation.

3. Monitor and Validate in Production:
- Implement real-time evaluation by collecting new image data from the field.
- Set up model drift detection to ensure accuracy remains high.

4. Future Considerations:
- Extend the model to detect other plant diseases.
- Build a mobile application for farmers to upload images and receive instant classification results.

---